# A Novel Deep Learning Approach for Wind Power Forecasting Based on WD-LSTM Model

## Import all the needed packages

In [1]:
# Basic Data Science Packages
import numpy as np
import pandas as pd

#TensorFlow packages required for building CNN + LSTM model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, BatchNormalization, Input, Dense, Flatten, LSTM, Reshape, TimeDistributed
from tensorflow.keras.models import Model

#Plotting graph
import matplotlib.pyplot as plt

#Plotting Keras Model
import keras.utils.vis_utils
from importlib import reload
reload(keras.utils.vis_utils)

from keras.utils.vis_utils import plot_model

from utility_functions import metrics

import warnings
warnings.filterwarnings("ignore")

## Performing Wavelength Decomposition

In [2]:
import pywt
x = [3, 7, 1, 1, -2, 5, 4, 6, -2, 5, 4, 6, -2, 5, 4, 6, -2, 5, 4, 6, -2, 5, 4, 6, -2, 5, 4, 6, -2, 5, 4, 6]
pywt.dwt(x, 'db2')

(array([5.65685425, 7.39923721, 0.22414387, 3.33677403, 5.85561413,
        3.33677403, 5.85561413, 3.33677403, 5.85561413, 3.33677403,
        5.85561413, 3.33677403, 5.85561413, 3.33677403, 5.85561413,
        3.33677403, 7.77817459]),
 array([-2.44948974, -1.60368225, -4.44140056, -0.41361256, -5.95034847,
        -0.41361256, -5.95034847, -0.41361256, -5.95034847, -0.41361256,
        -5.95034847, -0.41361256, -5.95034847, -0.41361256, -5.95034847,
        -0.41361256,  1.22474487]))

In [3]:
print(len(x))
len(pywt.dwt(x, 'db2')[1])

32


17

In [12]:
import pywt
wavelet = pywt.Wavelet('db3')
print(wavelet)

Wavelet db3
  Family name:    Daubechies
  Short name:     db
  Filters length: 6
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False


In [10]:
x = [3, 7, 1, 1, -2, 5, 4, 6]
db2 = pywt.Wavelet('db2')
X = pywt.wavedec(x, db2)
X

[array([5.65685425, 7.39923721, 0.22414387, 3.33677403, 7.77817459]),
 array([-2.44948974, -1.60368225, -4.44140056, -0.41361256,  1.22474487])]

In [6]:
X2 = pywt.wavedec(x, db2, level=10)
X2

[array([108.42189839, 108.48005619, 106.57278601, 113.69667873,
        121.20902534]),
 array([ 0.01138871, -2.45274748,  2.93540905, -1.77828066,  1.28423038]),
 array([-0.00668479, -1.86002085,  2.2189208 , -1.34506405,  0.99284889]),
 array([ 8.39805761e-04, -1.41408986e+00,  1.67441608e+00, -1.00804929e+00,
         7.46883270e-01]),
 array([ 0.06754315, -0.89705862,  1.28987116, -0.88619148,  0.4258358 ]),
 array([-0.1742078 , -0.96806557,  1.05471817, -0.65321314,  0.74076835]),
 array([ 0.50818811,  0.3891746 ,  0.98511537, -1.30479634, -0.57768174]),
 array([-0.31575022,  0.67321982,  1.73179787, -2.73248675,  0.64321928]),
 array([ 0.88547292,  3.74601388,  3.77237524, -7.25545535, -1.1484067 ]),
 array([-0.42770133,  3.72423411,  0.48210099,  1.06367045, -5.0083641 ]),
 array([-2.11206142, -2.64704675, -3.16825651, -0.67715519,  0.56811154,
         2.70377533])]